In [109]:
import pandas as pd
import tensorflow as tf
import numpy as np

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [110]:
DATAPATH = './data/games.csv'

In [111]:
games = pd.read_csv(DATAPATH)

In [112]:
games.dtypes

Unnamed: 0             int64
SEASON_ID              int64
TEAM_ID                int64
TEAM_ABBREVIATION     object
TEAM_NAME             object
GAME_ID                int64
GAME_DATE             object
MATCHUP               object
WL                    object
MIN                    int64
PTS                    int64
FGM                    int64
FGA                    int64
FG_PCT               float64
FG3M                   int64
FG3A                   int64
FG3_PCT              float64
FTM                    int64
FTA                    int64
FT_PCT               float64
OREB                   int64
DREB                   int64
REB                    int64
AST                    int64
STL                    int64
BLK                    int64
TOV                    int64
PF                     int64
PLUS_MINUS           float64
dtype: object

In [113]:
games = games.drop(['Unnamed: 0'], axis=1)

In [114]:
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22019,1610612748,MIA,Miami Heat,21900970,2020-03-11,MIA vs. CHA,L,239,98,...,0.833,9,27,36,32,8,5,14,17,-13.0
1,22019,1610612743,DEN,Denver Nuggets,21900973,2020-03-11,DEN @ DAL,L,240,97,...,0.625,3,37,40,23,6,4,15,20,-7.8
2,22019,1610612752,NYK,New York Knicks,21900969,2020-03-11,NYK @ ATL,W,264,136,...,0.735,6,41,47,32,14,9,12,23,0.0
3,22019,1610612742,DAL,Dallas Mavericks,21900973,2020-03-11,DAL vs. DEN,W,240,113,...,0.739,13,39,52,23,9,2,10,14,8.0
4,22019,1610612765,DET,Detroit Pistons,21900971,2020-03-11,DET @ PHI,L,241,106,...,0.680,6,26,32,25,10,3,8,24,-14.2


In [115]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('WL')
#     ds = tf.data.Dataset.from_tensor_slices((dataframe.values, labels.values))
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size = len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [116]:
games.pop('SEASON_ID')
games.pop('TEAM_ID')
games.pop('GAME_ID')
games.pop('TEAM_ABBREVIATION')

0        MIA
1        DEN
2        NYK
3        DAL
4        DET
        ... 
29995    BOS
29996    LAL
29997    ATL
29998    DAL
29999    DEN
Name: TEAM_ABBREVIATION, Length: 30000, dtype: object

In [117]:
feature_columns = []
obj_columns = ['TEAM_NAME', 'GAME_DATE', 'MATCHUP', 'WL']
#numeric columns
for i in obj_columns:
    games[i] = pd.Categorical(games[i])
    games[i] = games[i].cat.codes 
for header in games.columns:
    games[header] = games[header].astype('float64')
    if header != 'WL':
        feature_columns.append(feature_column.numeric_column(header))

In [118]:
games.dtypes

TEAM_NAME     float64
GAME_DATE     float64
MATCHUP       float64
WL            float64
MIN           float64
PTS           float64
FGM           float64
FGA           float64
FG_PCT        float64
FG3M          float64
FG3A          float64
FG3_PCT       float64
FTM           float64
FTA           float64
FT_PCT        float64
OREB          float64
DREB          float64
REB           float64
AST           float64
STL           float64
BLK           float64
TOV           float64
PF            float64
PLUS_MINUS    float64
dtype: object

In [119]:
games.head(n=250)

,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42.0,2427.0,1050.0,0.0,239.0,98.0,39.0,84.0,0.464,15.0,...,0.833,9.0,27.0,36.0,32.0,8.0,5.0,14.0,17.0,-13.0
1,22.0,2427.0,465.0,0.0,240.0,97.0,39.0,83.0,0.470,14.0,...,0.625,3.0,37.0,40.0,23.0,6.0,4.0,15.0,20.0,-7.8
2,52.0,2427.0,1395.0,1.0,264.0,136.0,50.0,94.0,0.532,11.0,...,0.735,6.0,41.0,47.0,32.0,14.0,9.0,12.0,23.0,0.0
3,21.0,2427.0,434.0,1.0,240.0,113.0,42.0,92.0,0.457,12.0,...,0.739,13.0,39.0,52.0,23.0,9.0,2.0,10.0,14.0,8.0
4,23.0,2427.0,546.0,0.0,241.0,106.0,39.0,84.0,0.464,11.0,...,0.680,6.0,26.0,32.0,25.0,10.0,3.0,8.0,24.0,-14.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,47.0,2411.0,1147.0,0.0,239.0,123.0,42.0,95.0,0.442,15.0,...,0.706,7.0,35.0,42.0,16.0,4.0,6.0,11.0,22.0,-16.0
246,41.0,2411.0,961.0,0.0,241.0,97.0,37.0,89.0,0.416,7.0,...,0.727,11.0,34.0,45.0,20.0,6.0,7.0,13.0,18.0,-27.0
247,20.0,2411.0,371.0,1.0,265.0,125.0,43.0,97.0,0.443,14.0,...,0.735,13.0,40.0,53.0,25.0,8.0,6.0,12.0,20.0,6.0
248,51.0,2410.0,1346.0,1.0,239.0,115.0,46.0,98.0,0.469,17.0,...,0.462,14.0,38.0,52.0,34.0,7.0,5.0,13.0,15.0,14.0


In [120]:
games = games.dropna()

In [121]:
print(feature_columns)

[NumericColumn(key='TEAM_NAME', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='GAME_DATE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='MATCHUP', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='MIN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PTS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FGM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FGA', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG_PCT', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG3M', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG3A', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key=

In [147]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
optimizer = tf.keras.optimizers.Adam(lr=0.001)
def get_compiled_model():
  model = tf.keras.Sequential([
    feature_layer,
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [148]:
train, test = train_test_split(games, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

19196 train examples
4800 validation examples
6000 test examples


In [149]:
batch_size = 128
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [150]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001) 
mc = ModelCheckpoint('best_model.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [151]:
model = get_compiled_model()
history = model.fit(train_ds, epochs=20, callbacks=[es,mc], validation_data=val_ds)
y_pred = model.predict(val_ds).round()
y_act = np.asarray(val['WL'])
y_act = y_act.astype('float64')
y_act = np.reshape(y_act, y_pred.shape)
print('Precision= ', precision_score(y_act, y_pred , average="macro", labels=np.unique(y_pred)))
print('Recall= ', recall_score(y_act, y_pred , average="macro", labels=np.unique(y_pred)))
print('Accuracy= ', history.history['val_accuracy'][-1])
print('Loss= ', history.history['val_loss'][-1])

Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

132/150 [=========================>....] - ETA: 0s - loss: 0.6084 - accuracy: 0.8039
Epoch 00001: val_accuracy improved from -inf to 0.79458, saving model to best_model.hdf5
150/150 [==============================] - 1s 5ms/step - loss: 0.6009 - accuracy: 0.8167 - val_loss: 0.5947 - val_accuracy: 0.7946
Epoch 2/20
140/150 [===========================>..] - ETA: 0s - loss: 0.5337 - accuracy: 0.9455
Epoch 00002: val_accuracy improved from 0.79458 to 0.97188, saving model to best_model.hdf5
150/

In [89]:
import sys
np.set_printoptions(threshold=sys.maxsize)
y_pred = model.predict(test_ds)
for i, j in zip(y_pred, test['WL']):
    print('predicted: ', i, 'actual: ', j)

predicted:  [1.] actual:  1.0
predicted:  [3.8640555e-05] actual:  0.0
predicted:  [1.1064698e-09] actual:  0.0
predicted:  [8.319249e-08] actual:  0.0
predicted:  [0.9999813] actual:  1.0
predicted:  [3.0090106e-09] actual:  0.0
predicted:  [0.99822176] actual:  1.0
predicted:  [0.99971247] actual:  1.0
predicted:  [0.9990505] actual:  1.0
predicted:  [2.3127322e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.99999857] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [9.0744884e-08] actual:  0.0
predicted:  [1.1421449e-11] actual:  0.0
predicted:  [0.02107534] actual:  0.0
predicted:  [0.99999976] actual:  1.0
predicted:  [1.2474709e-07] actual:  0.0
predicted:  [0.00010607] actual:  0.0
predicted:  [0.01484466] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.99999964] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [5.8954765e-11] actual:  0.0
predicted:  [0

predicted:  [0.9960535] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [3.5823577e-09] actual:  0.0
predicted:  [0.99998635] actual:  1.0
predicted:  [6.7067565e-09] actual:  0.0
predicted:  [0.00195149] actual:  0.0
predicted:  [0.0002805] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.99999845] actual:  1.0
predicted:  [9.2082614e-10] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [4.9128203e-06] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [3.9789305e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.0001815] actual:  0.0
predicted:  [2.7127274e-09] actual:  0.0
predicted:  [7.916706e-08] actual:  0.0
predicted:  [1.6865126e-08] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.99999964] actual:  1.0
predicted:  [3.3814172e-06] actual:  0.0
predicted:  [1.3748226e-12] actual:  0.0
predicted:  [0.00016204] actual:  0.0
predicted:  [2.2534698e-06] actual:  0.0
predicted:  [1.9719586e-08] actual:  0.

predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.3185439e-10] actual:  0.0
predicted:  [4.505897e-10] actual:  0.0
predicted:  [7.361132e-08] actual:  0.0
predicted:  [6.3696226e-10] actual:  0.0
predicted:  [9.366335e-05] actual:  0.0
predicted:  [1.8879212e-09] actual:  0.0
predicted:  [0.9999995] actual:  1.0
predicted:  [1.3304984e-10] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.6847733e-06] actual:  0.0
predicted:  [9.192473e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.01098704] actual:  0.0
predicted:  [3.6010188e-09] actual:  0.0
predicted:  [0.98738503] actual:  1.0
predicted:  [9.372352e-08] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [2.188774e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [2.2025607e-07] actual:  0.0
predicted:  [0.08197477] actual:  0.0
predicted:  [4.685251e-09] actual:  0.0
predicted:  [1.0368269e-06] actual:  0.0
predicted:  [1.] actual:  1.0
p

predicted:  [6.2217413e-09] actual:  0.0
predicted:  [1.5730194e-08] actual:  0.0
predicted:  [4.993056e-10] actual:  0.0
predicted:  [0.9999927] actual:  1.0
predicted:  [0.99563164] actual:  1.0
predicted:  [2.7045477e-10] actual:  0.0
predicted:  [1.4629496e-12] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.2475958e-07] actual:  0.0
predicted:  [0.99999976] actual:  1.0
predicted:  [1.115391e-08] actual:  0.0
predicted:  [9.372294e-09] actual:  0.0
predicted:  [1.1191836e-09] actual:  0.0
predicted:  [5.8173984e-05] actual:  0.0
predicted:  [3.5846817e-09] actual:  0.0
predicted:  [0.9999995] actual:  1.0
predicted:  [3.878198e-09] actual:  0.0
predicted:  [1.01172425e-07] actual:  0.0
predicted:  [2.904991e-09] actual:  0.0
predicted:  [2.0909912e-08] actual:  0.0
predicted:  [6.7118466e-10] actual:  0.0
predicted:  [1.7125641e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predict

predicted:  [1.10293e-09] actual:  0.0
predicted:  [2.098966e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.99992657] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [3.5163552e-08] actual:  0.0
predicted:  [3.249797e-06] actual:  0.0
predicted:  [4.249857e-09] actual:  0.0
predicted:  [5.635388e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.3708923e-08] actual:  0.0
predicted:  [7.39169e-08] actual:  0.0
predicted:  [1.5644342e-09] actual:  0.0
predicted:  [0.04759601] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [2.2014937e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999862] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.1253533e-08] actual:  0.0
predicted:  [3.8936e-09] actual:  0.0
predicted:  [4.0037598e-11] actual:  0.0
predicted:  [5.292163e-08] actual: 

...

predicted:  [2.0058083e-05] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [2.3563596e-09] actual:  0.0
predicted:  [0.99999976] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.99999917] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.7788838e-09] actual:  0.0
predicted:  [3.2359708e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [2.9234783e-09] actual:  0.0
predicted:  [0.99999917] actual:  1.0
predicted:  [0.9999995] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999982] actual:  1.0
predicted:  [2.483644e-08] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [5.9881204e-06] actual:  0.0
predicted:  [5.3954125e-07] actual:  0.0
predicted:  [1.2868956e-07] actual:  0.0
predicted:  [1.5476427e-08] actual:  0.0
predicted:  [1.5353141e-09] actual:  0.0
predicted:  [7.157681e-10] actua

predicted:  [1.] actual:  1.0
predicted:  [9.85501e-06] actual:  0.0
predicted:  [3.8763663e-08] actual:  0.0
predicted:  [1.15994204e-07] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.2618995e-08] actual:  0.0
predicted:  [0.99996006] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [8.3306935e-09] actual:  0.0
predicted:  [2.3654647e-09] actual:  0.0
predicted:  [6.28045e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [6.908499e-08] actual:  0.0
predicted:  [0.99482054] actual:  1.0
predicted:  [0.999701] actual:  1.0
predicted:  [3.6630134e-08] actual:  0.0
predicted:  [3.1283943e-08] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.5773818e-08] actual:  0.0
predicted:  [8.589467e-09] actual:  0.0
predicted:  [1.0323204e-07] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.99996793] actual:  1.0
predicted:  [1.9252966e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [5.078577e-10] actual:  0.0
p

predicted:  [1.] actual:  1.0
predicted:  [7.238218e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [2.2999073e-07] actual:  0.0
predicted:  [3.382265e-08] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [2.49639e-09] actual:  0.0
predicted:  [1.7873728e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [5.153029e-09] actual:  0.0
predicted:  [2.816609e-08] actual:  0.0
predicted:  [0.9983817] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.9995597] actual:  1.0
predicted:  [9.911768e-10] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [3.781352e-12] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.2485039e-09] actual:  0.0
predicted:  [8.514889e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [7.0259034e-08] actual:  0.0
predicted:  [0.9878632] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [2.4111868e-08] actual:  0.0
predicted:  [0.99998367] actual:  1.0
predicted:  [1.] actual:  1

predicted:  [0.99998957] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [2.625746e-10] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [6.7796975e-08] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [5.0885934e-10] actual:  0.0
predicted:  [0.9999981] actual:  1.0
predicted:  [0.99920464] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.99999714] actual:  1.0
predicted:  [1.1576058e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [0.99998796] actual:  1.0
predicted:  [0.99993527] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [2.6071145e-08] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [3.9025404e-11] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [2.4119228e-08] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999987] actual:  1.0
predicted:  [1.6782851e-06] actual:  0.0
predicted:  [9.845563e-09] actual:  0.0
predicted

predicted:  [1.] actual:  1.0
predicted:  [1.8815942e-09] actual:  0.0
predicted:  [5.8953247e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [4.537238e-06] actual:  0.0
predicted:  [1.7270356e-10] actual:  0.0
predicted:  [3.385854e-08] actual:  0.0
predicted:  [8.905801e-11] actual:  0.0
predicted:  [1.4904587e-06] actual:  0.0
predicted:  [0.00167793] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [0.9999639] actual:  1.0
predicted:  [1.] actual:  1.0
predicted:  [2.8698766e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [2.7774356e-06] actual:  0.0
predicted:  [5.5163856e-09] actual:  0.0
predicted:  [0.99999976] actual:  1.0
predicted:  [1.1212562e-09] actual:  0.0
predicted:  [1.] actual:  1.0
predicted:  [4.747553e-10] actual:  0.0
predicted:  [0.9999962] actual:  1.0
pr